In [1]:
import os
import argparse
from ml_collections import ConfigDict
import yaml
import time

import copy
import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch_geometric.transforms import Compose
from tqdm import tqdm
import wandb
import numpy as np
import random

from data.data_preprocess import HeteroAddLaplacianEigenvectorPE, SubSample, SubSample_
from data.dataset import LPDataset_harp, LPDataset
from data.utils import args_set_bool, collate_fn_ip_harp, collate_fn_ip
from models.hetero_gnn import QuadpartiteHeteroGNN, TripartiteHeteroGNN_
from trainer import Trainer

In [2]:
if wandb.run is not None:
    print('wandb running')
else:
    print('wandb not running')

wandb not running


In [3]:
# 2*ipm_steps/num_conv_layers = outer loop
# num_conv_layers = inner loop
var_dict = {
            "weight_decay": 0,
            "micro_batch": 16,         # oringal: 4, 
            "batchsize": 32,          # oringal: 128,
            "hidden": 180, 
            "num_conv_layers": 2,     # oringinal: 8,  
            "num_pred_layers": 4, 
            "num_mlp_layers": 4, 
#             "share_lin_weight": 'false', 
            "share_lin_weight": True, 
#             "conv_sequence": 'ceov', 
            "conv_sequence": 'cov', 
            "loss_weight_x": 1.0, 
            "loss_weight_obj": 3.43, 
            "loss_weight_cons": 5.8,     # oringinal: 5.8
            "losstype": 'l2',
            "runs": 3,
            "lappe": 0, 
            "conv": 'gcnconv', 
            "epoch": 50,    # oringal: 500
#             "ipm_alpha": 0.35,
            "ipm_alpha": 0.7,
    
            "ipm_steps": 16,       # 8 for supervising every mlp layer, 1 for supervising last layer
    
            "dropout": 0,
#             "share_conv_weight": 'false',        # default: false   
            "share_conv_weight": True,        # default: false  
#             "use_norm": 'true',
#             "use_res": 'false',  
            "use_norm": True,
            "use_res": True,  
            "lr": 1.e-5,     # original: 1.e-3
            "weight_decay": 0
            }

In [4]:
# wandb.init(project='reallocation_harp',
#            config=var_dict,
#            entity="fzhou")

In [5]:
if wandb.run is not None:
    print('wandb running')
else:
    print('wandb not running')

wandb not running


In [6]:
# -Ax+b<=-1
# Use TripartitionGNN

trainleq1_ins = 'trainleq1_abilene' 
validleq1_ins = 'validleq1_abilene'
testleq1_ins = 'testleq1_abilene'
ipm = 16


trainleq1_dataset = LPDataset('raw',
                    extra_path=f'{1}restarts_'
                                     f'{0}lap_'
                                     f'{ipm}steps'
                                     f'{"_upper_" + str(trainleq1_ins)}',
                    upper_bound=1,
                    rand_starts=1,
                    pre_transform=Compose([HeteroAddLaplacianEigenvectorPE(k=0),
                                                 SubSample(ipm)]))
validleq1_dataset = LPDataset('raw',
                    extra_path=f'{1}restarts_'
                                     f'{0}lap_'
                                     f'{ipm}steps'
                                     f'{"_upper_" + str(validleq1_ins)}',
                    upper_bound=1,
                    rand_starts=1,
                    pre_transform=Compose([HeteroAddLaplacianEigenvectorPE(k=0),
                                                 SubSample(ipm)]))
testleq1_dataset = LPDataset('raw',
                    extra_path=f'{1}restarts_'
                                     f'{0}lap_'
                                     f'{ipm}steps'
                                     f'{"_upper_" + str(testleq1_ins)}',
                    upper_bound=1,
                    rand_starts=1,
                    pre_transform=Compose([HeteroAddLaplacianEigenvectorPE(k=0),
                                                 SubSample(ipm)]))


train_loader = DataLoader(trainleq1_dataset,
                          batch_size=var_dict['batchsize'],
                          shuffle=True,
                          num_workers=1,
                          collate_fn=collate_fn_ip)
val_loader = DataLoader(validleq1_dataset,
                        batch_size=var_dict['batchsize'],
                        shuffle=True,
                        num_workers=1,
                        collate_fn=collate_fn_ip)
test_loader = DataLoader(testleq1_dataset,
                        batch_size=var_dict['batchsize'],
                        shuffle=False,
                        num_workers=1,
                        collate_fn=collate_fn_ip)

In [7]:
trainleq1_dataset, validleq1_dataset, testleq1_dataset

(LPDataset(12095), LPDataset(2016), LPDataset(2016))

In [8]:
len(trainleq1_dataset)/var_dict["batchsize"], len(trainleq1_dataset)%var_dict["batchsize"]

(377.96875, 31)

In [9]:
len(validleq1_dataset)/var_dict["batchsize"], len(validleq1_dataset)%var_dict["batchsize"]

(63.0, 0)

In [10]:
len(testleq1_dataset)/var_dict["batchsize"], len(testleq1_dataset)%var_dict["batchsize"]

(63.0, 0)

In [11]:
len(train_loader), len(val_loader), len(test_loader)

(378, 63, 63)

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Tri

In [13]:
# testing case
# TripartiteHeteroGNN_, train_newnew_con: return primal, obj, con loss 

seed = 2026     # 2025, 2026, 2027, 2028
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

best_val_objgap_mean = []
best_val_consgap_mean = []
test_objgap_mean = []
test_consgap_mean = []
test_objgap_nocon_mean = []

for run in range(1):
    
    if not os.path.isdir('logs'):
        os.mkdir('logs')
    exist_runs = [d for d in os.listdir('logs') if d.startswith('exp')]
    log_folder_name = f'logs/exp{len(exist_runs)}'
    os.mkdir(log_folder_name)
    with open(os.path.join(log_folder_name, 'config.yaml'), 'w') as outfile:
        yaml.dump(var_dict, outfile, default_flow_style=False)
            
    os.mkdir(os.path.join(log_folder_name, f'run{run}'))

    model = TripartiteHeteroGNN_(ipm_steps=var_dict['ipm_steps'],
                                 conv=var_dict['conv'],
                                 in_shape=2,
                                 pe_dim=var_dict['lappe'],
                                 hid_dim=var_dict['hidden'],
                                 num_conv_layers=var_dict['num_conv_layers'],
                                 num_pred_layers=var_dict['num_pred_layers'],
                                 num_mlp_layers=var_dict['num_mlp_layers'],
                                 dropout=var_dict['dropout'],
                                 share_conv_weight=var_dict['share_conv_weight'],
                                 share_lin_weight=var_dict['share_lin_weight'],
                                 use_norm=var_dict['use_norm'],
                                 use_res=var_dict['use_res'],
                                 conv_sequence=var_dict['conv_sequence']).to(device)
    
    best_model = copy.deepcopy(model.state_dict())

    optimizer = optim.Adam(model.parameters(), lr=var_dict['lr'], weight_decay=var_dict['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1.e-6)

    trainer = Trainer(device,
                      'primal+objgap+constraint',
                      var_dict['losstype'],
                      var_dict['micro_batch'],
                      var_dict['ipm_steps'],
                      var_dict['ipm_alpha'],
                      loss_weight={'primal': var_dict['loss_weight_x'],
                                   'objgap': var_dict['loss_weight_obj'],
                                   'constraint': var_dict['loss_weight_cons']})

    pbar = tqdm(range(var_dict['epoch']))
    curr = time.time()
    for epoch in pbar:
        train_loss, primal_loss, obj_loss, cons_loss = trainer.train_newnewcon(train_loader, model, optimizer)
        with torch.no_grad():

            val_gaps, val_constraint_gap = trainer.eval_metrics(val_loader, model)

            # metric to cache the best model
            cur_mean_gap = val_gaps[:, -1].mean().item()
            cur_cons_gap_mean = val_constraint_gap[:, -1].mean().item()
            if scheduler is not None:
                scheduler.step(cur_mean_gap)
                
            torch.save(model.state_dict(), os.path.join(log_folder_name, f'run{run}', str(epoch)+'_model.pt'))
            
            if trainer.best_val_objgap > cur_mean_gap:
                trainer.patience = 0
                trainer.best_val_objgap = cur_mean_gap
                trainer.best_val_consgap = cur_cons_gap_mean
                best_model = copy.deepcopy(model.state_dict())
  
                torch.save(model.state_dict(), os.path.join(log_folder_name, f'run{run}', str(epoch)+'_best_model.pt'))


        pbar.set_postfix({'train_loss': train_loss,
                          'primal_loss': primal_loss,
                          'obj_loss': obj_loss,
                          'cons_loss': cons_loss,
                          'val_obj': cur_mean_gap,
                          'val_cons': cur_cons_gap_mean,
                          'lr': scheduler.optimizer.param_groups[0]["lr"]})
        log_dict = {'train_loss': train_loss,
                    'primal_loss': primal_loss,
                    'obj_loss': obj_loss,
                    'cons_loss': cons_loss,
                    'val_obj_gap_last_mean': cur_mean_gap,
                    'val_cons_gap_last_mean': cur_cons_gap_mean,
                   'lr': scheduler.optimizer.param_groups[0]["lr"]}

        wandb.log(log_dict)

    print('time:', time.time()-curr)

    best_val_objgap_mean.append(trainer.best_val_objgap)
    best_val_consgap_mean.append(trainer.best_val_consgap)

    model.load_state_dict(best_model)
    
    with torch.no_grad():

        test_gaps, test_cons_gap, test_gaps_nocon = trainer.eval_metrics_(test_loader, model)

    
    test_objgap_mean.append(test_gaps[:, -1].mean().item())
    test_consgap_mean.append(test_cons_gap[:, -1].mean().item())
    test_objgap_nocon_mean.append(test_gaps_nocon[:, -1].mean().item())

    wandb.log({'test_objgap': test_objgap_mean[-1]})
    wandb.log({'test_consgap': test_consgap_mean[-1]})
    wandb.log({'test_objgap_nocon': test_objgap_nocon_mean[-1]})


wandb.log({
    'best_val_objgap': np.mean(best_val_objgap_mean),
    'test_objgap_mean': np.mean(test_objgap_mean),
    'test_objgap_std': np.std(test_objgap_mean),
    'test_consgap_mean': np.mean(test_consgap_mean),
    'test_consgap_std': np.std(test_consgap_mean),
    'test_hybrid_gap': np.mean(test_objgap_mean) + np.mean(test_consgap_mean),  # for the sweep
})
wandb.finish()


  0%|                                                    | 0/50 [00:00<?, ?it/s]

-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0')
x[1] torch.Size([32, 360])
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([32, 36

row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0')
x[1] torch.Size([33824, 360])
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 3

row, col: tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0')
x[1] torch.Size([32, 360])
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], devic

row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0')
x[1] torch.Size([33824, 360])
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 3

...

-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([33824, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0')
x[1] torch.Size([33824, 360])
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32

row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0')
x[1] torch.Size([32, 360])
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0')
x[1] torch.Size([32, 360])
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([33824, 360])
row, col: tensor([   0,    0,    0,  ..., 5183, 5183, 5183], device='cuda:0') tensor([    0,     1,     2,  ..., 33814, 33818, 33823], dev

  0%|                                                    | 0/50 [00:03<?, ?it/s]

tensor([    0,     0,     1,  ..., 33823, 33823, 33823], device='cuda:0') tensor([   0,  132,    0,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([5184, 360])
row, col: tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0') tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0')
x[1] torch.Size([5184, 360])
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([33824, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([    0,     1,     2,  ..., 33821, 33822, 33823], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0')
x[1] torch.Size([32, 360])
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([32, 360])
row, col: tensor([   0,    1,    2,  ..., 5181, 5182, 5183], device='cuda:0') tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0'

KeyboardInterrupt: 

In [13]:
# TripartiteHeteroGNN_, train_newnew_con: return primal, obj, con loss 

seed = 2026     # 2025, 2026, 2027, 2028
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

best_val_objgap_mean = []
best_val_consgap_mean = []
test_objgap_mean = []
test_consgap_mean = []
test_objgap_nocon_mean = []

for run in range(1):
    
    if not os.path.isdir('logs'):
        os.mkdir('logs')
    exist_runs = [d for d in os.listdir('logs') if d.startswith('exp')]
    log_folder_name = f'logs/exp{len(exist_runs)}'
    os.mkdir(log_folder_name)
    with open(os.path.join(log_folder_name, 'config.yaml'), 'w') as outfile:
        yaml.dump(var_dict, outfile, default_flow_style=False)
            
    os.mkdir(os.path.join(log_folder_name, f'run{run}'))

    model = TripartiteHeteroGNN_(ipm_steps=var_dict['ipm_steps'],
                                 conv=var_dict['conv'],
                                 in_shape=2,
                                 pe_dim=var_dict['lappe'],
                                 hid_dim=var_dict['hidden'],
                                 num_conv_layers=var_dict['num_conv_layers'],
                                 num_pred_layers=var_dict['num_pred_layers'],
                                 num_mlp_layers=var_dict['num_mlp_layers'],
                                 dropout=var_dict['dropout'],
                                 share_conv_weight=var_dict['share_conv_weight'],
                                 share_lin_weight=var_dict['share_lin_weight'],
                                 use_norm=var_dict['use_norm'],
                                 use_res=var_dict['use_res'],
                                 conv_sequence=var_dict['conv_sequence']).to(device)
    
    best_model = copy.deepcopy(model.state_dict())

    optimizer = optim.Adam(model.parameters(), lr=var_dict['lr'], weight_decay=var_dict['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1.e-6)

    trainer = Trainer(device,
                      'primal+objgap+constraint',
                      var_dict['losstype'],
                      var_dict['micro_batch'],
                      var_dict['ipm_steps'],
                      var_dict['ipm_alpha'],
                      loss_weight={'primal': var_dict['loss_weight_x'],
                                   'objgap': var_dict['loss_weight_obj'],
                                   'constraint': var_dict['loss_weight_cons']})

    pbar = tqdm(range(var_dict['epoch']))
    curr = time.time()
    for epoch in pbar:
        train_loss, primal_loss, obj_loss, cons_loss = trainer.train_newnewcon(train_loader, model, optimizer)
        with torch.no_grad():

            val_gaps, val_constraint_gap = trainer.eval_metrics(val_loader, model)

            # metric to cache the best model
            cur_mean_gap = val_gaps[:, -1].mean().item()
            cur_cons_gap_mean = val_constraint_gap[:, -1].mean().item()
            if scheduler is not None:
                scheduler.step(cur_mean_gap)
                
            torch.save(model.state_dict(), os.path.join(log_folder_name, f'run{run}', str(epoch)+'_model.pt'))
            
            if trainer.best_val_objgap > cur_mean_gap:
                trainer.patience = 0
                trainer.best_val_objgap = cur_mean_gap
                trainer.best_val_consgap = cur_cons_gap_mean
                best_model = copy.deepcopy(model.state_dict())
  
                torch.save(model.state_dict(), os.path.join(log_folder_name, f'run{run}', str(epoch)+'_best_model.pt'))


        pbar.set_postfix({'train_loss': train_loss,
                          'primal_loss': primal_loss,
                          'obj_loss': obj_loss,
                          'cons_loss': cons_loss,
                          'val_obj': cur_mean_gap,
                          'val_cons': cur_cons_gap_mean,
                          'lr': scheduler.optimizer.param_groups[0]["lr"]})
        log_dict = {'train_loss': train_loss,
                    'primal_loss': primal_loss,
                    'obj_loss': obj_loss,
                    'cons_loss': cons_loss,
                    'val_obj_gap_last_mean': cur_mean_gap,
                    'val_cons_gap_last_mean': cur_cons_gap_mean,
                   'lr': scheduler.optimizer.param_groups[0]["lr"]}

        wandb.log(log_dict)

    print('time:', time.time()-curr)

    best_val_objgap_mean.append(trainer.best_val_objgap)
    best_val_consgap_mean.append(trainer.best_val_consgap)

    model.load_state_dict(best_model)
    
    with torch.no_grad():

        test_gaps, test_cons_gap, test_gaps_nocon = trainer.eval_metrics_(test_loader, model)

    
    test_objgap_mean.append(test_gaps[:, -1].mean().item())
    test_consgap_mean.append(test_cons_gap[:, -1].mean().item())
    test_objgap_nocon_mean.append(test_gaps_nocon[:, -1].mean().item())

    wandb.log({'test_objgap': test_objgap_mean[-1]})
    wandb.log({'test_consgap': test_consgap_mean[-1]})
    wandb.log({'test_objgap_nocon': test_objgap_nocon_mean[-1]})


wandb.log({
    'best_val_objgap': np.mean(best_val_objgap_mean),
    'test_objgap_mean': np.mean(test_objgap_mean),
    'test_objgap_std': np.std(test_objgap_mean),
    'test_consgap_mean': np.mean(test_consgap_mean),
    'test_consgap_std': np.std(test_consgap_mean),
    'test_hybrid_gap': np.mean(test_objgap_mean) + np.mean(test_consgap_mean),  # for the sweep
})
wandb.finish()


100%|█| 50/50 [5:16:26<00:00, 379.73s/it, train_loss=0.0261, primal_loss=0.00781

time: 18986.45188331604


before nocon tensor([[-4.2427e-01, -5.6015e-01, -4.1584e-01, -3.8651e-01, -1.2923e-02,
          2.5063e-01,  1.3260e-01,  3.3698e-02,  5.9848e-02,  9.9728e-03,
         -1.8431e-02, -6.3021e-03,  1.1734e-02,  1.4688e-02,  8.3803e-03,
          4.0407e-03],
        [-4.2430e-01, -5.6436e-01, -4.2086e-01, -3.8698e-01, -3.9070e-01,
          1.7594e-02,  7.6977e-02,  1.4619e-02,  7.8821e-02,  7.5675e-02,
          5.5666e-02,  8.1663e-02,  8.4452e-02,  1.0731e-01,  1.1927e-01,
          1.2262e-01],
        [-4.2375e-01, -5.5375e-01, -4.1614e-01, -3.8525e-01,  1.6768e-02,
          3.1205e-01,  2.2034e-01,  8.5279e-02,  1.0212e-01,  6.2260e-02,
          2.8808e-02,  6.0043e-02,  8.4403e-02,  8.8093e-02,  8.8604e-02,
          9.2943e-02],
        [-4.2419e-01, -5.5938e-01, -4.1858e-01, -3.8636e-01, -3.2488e-02,
          2.6581e-01,  1.5946e-01,  6.7625e-02,  9.8798e-02,  4.9526e-02,
          2.8394e-02,  4.6000e-02,  6.6658e-02,  7.4066e-02,  7.3113e-02,
          6.9494e-02],
       

before nocon tensor([[-4.2396e-01, -5.5169e-01, -4.2015e-01, -3.8646e-01,  3.7781e-02,
          3.7940e-01,  2.0108e-01,  1.0645e-01,  1.2907e-01,  7.6027e-02,
          3.7253e-02,  5.0533e-02,  5.4764e-02,  4.0767e-02,  5.0714e-02,
          5.1274e-02],
        [-4.2404e-01, -5.5347e-01, -4.2068e-01, -3.8647e-01,  3.4694e-02,
          3.9108e-01,  1.8567e-01,  9.4477e-02,  1.2949e-01,  7.7421e-02,
          4.2559e-02,  6.0751e-02,  6.8193e-02,  5.8287e-02,  6.5214e-02,
          6.9439e-02],
        [-4.2372e-01, -5.5289e-01, -4.1920e-01, -3.8536e-01,  4.0352e-02,
          4.0841e-01,  2.4698e-01,  1.4102e-01,  1.6958e-01,  1.2515e-01,
          9.8345e-02,  1.1805e-01,  1.3916e-01,  1.4577e-01,  1.4327e-01,
          1.4095e-01],
        [-4.2390e-01, -5.5168e-01, -4.1629e-01, -3.8659e-01, -2.7998e-02,
          2.6275e-01,  2.1400e-01,  9.8745e-02,  1.1932e-01,  7.1627e-02,
          3.5642e-02,  4.9817e-02,  5.2025e-02,  3.9494e-02,  4.3744e-02,
          4.0840e-02],
       

before nocon tensor([[-4.2386e-01, -5.4157e-01, -4.1063e-01, -3.8760e-01, -1.8461e-02,
          2.1280e-01,  1.9792e-01,  8.6029e-02,  9.3640e-02,  4.2401e-02,
          6.2527e-03,  1.7632e-02,  2.8604e-02,  7.6207e-03,  2.1932e-02,
          2.0472e-02],
        [-4.2389e-01, -5.4212e-01, -4.0664e-01, -3.8711e-01, -7.7632e-03,
          2.3127e-01,  2.2863e-01,  1.1000e-01,  1.2312e-01,  7.1390e-02,
          3.5934e-02,  4.9106e-02,  5.5204e-02,  3.0804e-02,  5.0008e-02,
          4.6015e-02],
        [-4.2377e-01, -5.3500e-01, -4.0471e-01, -3.8698e-01, -1.3854e-02,
          2.1810e-01,  2.2708e-01,  1.1179e-01,  1.2666e-01,  7.6138e-02,
          3.7218e-02,  5.8236e-02,  6.0324e-02,  3.4410e-02,  5.6866e-02,
          4.3465e-02],
        [-4.2364e-01, -5.3997e-01, -4.0993e-01, -3.8686e-01, -3.1321e-02,
          1.8203e-01,  1.9284e-01,  7.7867e-02,  8.0006e-02,  3.2636e-02,
         -7.6245e-03,  9.4446e-03,  2.0704e-02, -6.9655e-03,  1.5708e-02,
          1.5036e-02],
       

...

before nocon tensor([[-4.2452e-01, -5.6916e-01, -4.2982e-01, -3.8813e-01, -4.0190e-01,
          8.6006e-03,  1.2313e-02, -3.3705e-02,  3.3028e-02,  1.0503e-02,
         -2.0375e-03,  2.0045e-02,  2.1273e-02,  4.2336e-02,  5.6179e-02,
          5.7547e-02],
        [-4.2468e-01, -5.7361e-01, -4.3493e-01, -3.9105e-01, -4.0628e-01,
         -3.6506e-02,  3.9050e-03, -2.2637e-02,  3.5494e-02,  1.6122e-02,
          3.9828e-03,  4.0248e-02,  4.1973e-02,  7.0785e-02,  8.5133e-02,
          8.3460e-02],
        [-4.2449e-01, -5.7100e-01, -4.3281e-01, -3.8969e-01, -4.0519e-01,
         -4.1373e-02, -6.9483e-03, -5.6020e-02, -3.4513e-03, -1.9068e-02,
         -3.2330e-02, -1.3462e-02, -1.2363e-02,  1.1437e-02,  2.2186e-02,
          2.0796e-02],
        [-4.2487e-01, -5.7826e-01, -4.3876e-01, -3.9275e-01, -4.0747e-01,
         -6.9988e-02, -3.8105e-02, -5.0025e-02,  1.5767e-02,  1.2759e-02,
         -1.9289e-03,  1.3660e-02,  2.2608e-02,  3.7666e-02,  4.0161e-02,
          3.6688e-02],
       

before nocon tensor([[-4.2524e-01, -5.9379e-01, -4.4484e-01, -3.9206e-01, -4.0447e-01,
         -1.1347e-01, -3.4052e-02, -7.2089e-02, -6.4319e-03,  2.4732e-02,
          3.1969e-02,  4.7587e-02,  4.4994e-02,  4.3013e-02,  4.9880e-02,
          5.4914e-02],
        [-4.2537e-01, -5.9334e-01, -4.4373e-01, -3.9324e-01, -4.0540e-01,
         -8.1260e-02, -1.6192e-02, -4.7680e-02,  3.3190e-03,  1.9137e-02,
          1.2771e-02,  2.6038e-02,  1.4248e-02,  9.4997e-03,  1.5886e-02,
          1.7720e-02],
        [-4.2536e-01, -5.9214e-01, -4.4596e-01, -3.9414e-01, -4.0543e-01,
         -7.4351e-02, -2.3668e-02, -6.2911e-02, -1.5323e-02, -4.3091e-03,
         -1.6389e-02, -1.3284e-02, -2.8151e-02, -3.7581e-02, -3.0402e-02,
         -2.9756e-02],
        [-4.2540e-01, -5.9299e-01, -4.5250e-01, -3.9422e-01, -4.0566e-01,
         -9.6090e-02, -3.0276e-02, -6.9454e-02, -7.8155e-03,  3.0631e-02,
          2.5904e-02,  3.7100e-02,  2.4396e-02,  1.6558e-02,  1.9671e-02,
          1.8058e-02],
       

before nocon tensor([[-4.2530e-01, -5.9080e-01, -4.4397e-01, -3.9154e-01, -4.0526e-01,
         -2.1343e-02, -2.9865e-02, -5.9730e-02, -7.0873e-03, -1.6950e-03,
         -1.6027e-02, -1.4084e-02, -2.8888e-02, -3.3522e-02, -2.3518e-02,
         -2.3823e-02],
        [-4.2534e-01, -5.9162e-01, -4.4420e-01, -3.9184e-01, -4.0512e-01,
         -5.7217e-02, -3.7791e-02, -6.2112e-02, -9.9789e-03, -2.6102e-03,
         -1.6053e-02, -1.1696e-02, -2.6315e-02, -3.5027e-02, -2.8528e-02,
         -2.8475e-02],
        [-4.2524e-01, -5.8951e-01, -4.4579e-01, -3.9195e-01, -4.0642e-01,
         -5.2867e-02, -6.7018e-02, -8.0631e-02, -2.7620e-02, -2.4610e-02,
         -3.7993e-02, -3.6390e-02, -5.1312e-02, -5.4329e-02, -4.5249e-02,
         -4.6753e-02],
        [-4.2529e-01, -5.8953e-01, -4.4363e-01, -3.9173e-01, -4.0700e-01,
         -3.8854e-02, -4.1898e-02, -7.5729e-02, -2.8742e-02, -2.7416e-02,
         -3.9505e-02, -4.1381e-02, -5.2149e-02, -4.4624e-02, -3.6855e-02,
         -4.1164e-02],
       

before nocon tensor([[-0.4250, -0.5847, -0.4365, -0.3900, -0.4043,  0.0215,  0.0302, -0.0197,
          0.0500,  0.0637,  0.0411,  0.0423,  0.0345,  0.0398,  0.0488,  0.0457],
        [-0.4250, -0.5868, -0.4350, -0.3897, -0.4033,  0.0126,  0.0215, -0.0239,
          0.0340,  0.0694,  0.0560,  0.0583,  0.0446,  0.0414,  0.0507,  0.0488],
        [-0.4251, -0.5865, -0.4321, -0.3896, -0.4040,  0.0127,  0.0361,  0.0064,
          0.0723,  0.0880,  0.0713,  0.0732,  0.0589,  0.0603,  0.0690,  0.0656],
        [-0.4251, -0.5904, -0.4366, -0.3895, -0.4020, -0.0092, -0.0089, -0.0668,
          0.0175,  0.0432,  0.0299,  0.0380,  0.0328,  0.0263,  0.0326,  0.0384],
        [-0.4251, -0.5903, -0.4377, -0.3896, -0.4033, -0.0156, -0.0242, -0.0708,
          0.0089,  0.0225,  0.0080,  0.0137,  0.0030, -0.0032,  0.0057,  0.0072],
        [-0.4253, -0.5891, -0.4376, -0.3910, -0.4064,  0.0360,  0.0122, -0.0058,
          0.0560,  0.0717,  0.0612,  0.0622,  0.0455,  0.0454,  0.0541,  0.0476],
        [

best_val_objgap,▁
cons_loss,▁▃▆█▄▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,█████████▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
obj_loss,█▆▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
primal_loss,█▆▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_consgap,▁
test_consgap_mean,▁
test_consgap_std,▁
test_hybrid_gap,▁
test_objgap,▁
test_objgap_mean,▁


In [ ]:
# telgen
# run time each epoch: 338.77078770160676 (s) 
# test time each instance: 18.935424737308335 (ms)

In [ ]:
# harp
# run time each epoch: 26.625265402793886 (s)
# test time each instance: 26.88961783571849 (ms)